# Sparseを実装する

本名 sparse vector algorithm. Dwork and Roth 2014を参照。

PRESLにおいてはjoint differentially privateにLPを解くために利用される。

気持ちとしては下のような感じ。

離散化した値の組合せそれぞれに対して、LPの解を計算したい。が、その出力には差分プライバシーを守っていてほしい。
ので、LPをそれぞれに対して解いた時に、その解を貯めておき、それをSparseにぶっこむことで出力にノイズを入れながら、その解の中で重要なものは一定の確率で得られているようにする。


なのでSparse自体は、入力であるqueryの結果にノイズを加え、出力するか否かのthresholdにもノイズを加えながら一定以上の値ならそれを出力する、というもの。

これは名前の通りthresholdを超える値が少ないと想定される時に有効なアルゴリズムで、この意味でsparseと名付けられている。

## 具体例

In [56]:
import numpy as np
from numpy.random import laplace
from numpy.random import uniform
np.random.seed(123)

In [57]:
datasize = 100
days = 1000
data = uniform(size=(100, 10000))

このデータから列ごとの平均値を取り出したいとする。

In [58]:
truevalues = data.mean(axis = 0)
truevalues

array([0.48800237, 0.49976037, 0.4840857 , ..., 0.46029493, 0.49353913,
       0.48631572])

In [53]:
truevalues.shape

(10000,)

これを引き出す時に何らかの事情でprivacyを保護したいとする。この時、「平均値を取り出す」というqueryに対してsparseを使う。

In [54]:
def sparse(data, T, num_answers, gamma, epsilon):
    
    datasize = data.shape[0]
    output = np.zeros(datasize)
    
    count = 0
    count2 = 0
    
    T_hat = T + laplace(scale = 2*gamma/epsilon)
    sigma = 2*num_answers*gamma/epsilon
    
    for i in range(datasize):
        
        nu = laplace(scale = sigma)
        Q_hat = data[i] + nu
        count2 += 1
        
        if Q_hat >= T_hat:
            
            output[i] = Q_hat
            count += 1
            
            if count >= num_answers:
                
                break
        
        else:
            
            output[i] = np.nan
    
    print(count)
    print(count2)
    
    return output[:count2],count2

この設定でsensitivity $\gamma$はどこか一箇所の$0$が$１$になる時の$\frac{1}{100}$

平均値が以上に高い時にそれを検知したい、というようなモチベーションだとしてthresholdを$T = 0.6$

$3$回検知したいとして、num_answers = 3

privacy parameter $\epsilon = 0.5$として、

In [59]:
gamma = 1/100
T = 0.6
num_answers = 3
epsilon = 0.5

In [37]:
result, num_query = sparse(data, T, num_answers, epsilon)

10
31


In [38]:
result

array([2.97266959, 2.37046394,        nan,        nan,        nan,
       1.30079236, 5.82016052, 5.94992694,        nan,        nan,
              nan,        nan,        nan, 3.45348201,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan, 1.88072118, 2.33305414,
              nan,        nan,        nan,        nan, 2.06746918,
       2.22699771])

In [42]:
beta = 0.99999
alpha = 4*num_answers*gamma*(np.log(num_query) + np.log(2*num_answers/beta))/epsilon
alpha

25.718917912356552

In [43]:
data[:num_query]

array([0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
       0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
       0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
       0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
       0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
       0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
       0.09210494])